In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.linalg import solve

In [3]:
dataPath = Path("/home/milo/workspace/pyplier/tests/data/common")
data = pd.read_csv(filepath_or_buffer = dataPath / "data.csv", index_col=0)

In [4]:
plierRes_b = pd.read_csv(filepath_or_buffer = dataPath / "plierRes_b.csv", index_col=0)
plierRes_c = pd.read_csv(filepath_or_buffer = dataPath / "plierRes_c.csv", index_col=0)
plierRes_residual = pd.read_csv(filepath_or_buffer = dataPath / "plierRes_residual.csv", index_col=0)
plierRes_u = pd.read_csv(filepath_or_buffer = dataPath / "plierRes_u.csv", index_col=0)
plierRes_z = pd.read_csv(filepath_or_buffer = dataPath / "plierRes_z.csv", index_col=0)
plierRes_heldoutgenes = pd.read_csv(filepath_or_buffer = dataPath / "heldOutGenes.csv", index_col=0)

priorMat = pd.read_csv(filepath_or_buffer=dataPath / "priorMat.csv", index_col=0)
pathwayMat = pd.read_csv(filepath_or_buffer=dataPath / "pathwayMat.csv", index_col=0)

In [7]:
plierRes = {
    "residual": plierRes_residual,
    "B": plierRes_b,
    "C": plierRes_c,
    "U": plierRes_u,
    "Z": plierRes_z,
    "L1": 18.5633,
    "L2": 37.12661,
    "L3": 0.0005530844,
    "heldOutGenes": {k: g["value"].tolist() for k, g in plierRes_heldoutgenes.groupby("name")},
}

In [10]:
ngenes = 50
auc_cutoff = 0.7
fdr_cutoff = 0.01

In [13]:
pathwayMat.head()

,IRIS_Bcell-Memory_IgG_IgA,IRIS_Bcell-Memory_IgM,IRIS_Bcell-naive,IRIS_CD4Tcell-N0,IRIS_CD4Tcell-Th1-restimulated12hour,IRIS_CD4Tcell-Th1-restimulated48hour,IRIS_CD4Tcell-Th2-restimulated12hour,IRIS_CD4Tcell-Th2-restimulated48hour,IRIS_CD8Tcell-N0,IRIS_DendriticCell-Control,...,KEGG_GNRH_SIGNALING_PATHWAY,KEGG_BASAL_TRANSCRIPTION_FACTORS,REACTOME_SYNTHESIS_OF_DNA,KEGG_HEMATOPOIETIC_CELL_LINEAGE,KEGG_T_CELL_RECEPTOR_SIGNALING_PATHWAY,PID_IL4_2PATHWAY,REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR,PID_BCR_5PATHWAY,PID_TELOMERASEPATHWAY,PID_PI3KPLCTRKPATHWAY
GAS6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
MMP14,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
MARCKSL1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SPARC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CTSD,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
plierRes["Z"].index

Index(['GAS6', 'MMP14', 'MARCKSL1', 'SPARC', 'CTSD', 'EPAS1', 'PALLD', 'PHC2',
       'LGALS3BP', 'SERPING1',
       ...
       'CDHR1', 'RRP12', 'SEC31B', 'GZMM', 'PLEKHF1', 'TRPM4', 'LAIR2',
       'ZNF135', 'MARCH3', 'SEPT8'],
      dtype='object', name='index', length=5964)

In [15]:
plierRes["U"].index

Index(['IRIS_Bcell-Memory_IgG_IgA', 'IRIS_Bcell-Memory_IgM',
       'IRIS_Bcell-naive', 'IRIS_CD4Tcell-N0',
       'IRIS_CD4Tcell-Th1-restimulated12hour',
       'IRIS_CD4Tcell-Th1-restimulated48hour',
       'IRIS_CD4Tcell-Th2-restimulated12hour',
       'IRIS_CD4Tcell-Th2-restimulated48hour', 'IRIS_CD8Tcell-N0',
       'IRIS_DendriticCell-Control',
       ...
       'KEGG_GNRH_SIGNALING_PATHWAY', 'KEGG_BASAL_TRANSCRIPTION_FACTORS',
       'REACTOME_SYNTHESIS_OF_DNA', 'KEGG_HEMATOPOIETIC_CELL_LINEAGE',
       'KEGG_T_CELL_RECEPTOR_SIGNALING_PATHWAY', 'PID_IL4_2PATHWAY',
       'REACTOME_SIGNALING_BY_THE_B_CELL_RECEPTOR_BCR', 'PID_BCR_5PATHWAY',
       'PID_TELOMERASEPATHWAY', 'PID_PI3KPLCTRKPATHWAY'],
      dtype='object', name='index', length=628)

In [11]:
pathwayMat_subset = pathwayMat.loc[plierRes["Z"].index, plierRes["U"].index]

KeyError: "['ATHL1', 'SERGEF', 'NAALADL1', 'FRMD8', 'SAMSN1', 'HOXA1', 'TRIB2', 'ZFP36L2', 'IL1RL1', 'SCN9A', 'FAM124B', 'SP140', 'CD300A', 'MICAL3', 'PIK3IP1', 'DEPDC5', 'APOL6', 'APOL3', 'UPK3A', 'TTC38', 'PLCH2', 'RCAN3', 'SMPDL3B', 'BEND5', 'CHI3L2', 'PBXIP1', 'SLAMF1', 'LY9', 'FAIM3', 'RAB27B', 'CCDC102B', 'GPR183', 'RASA3', 'FBXL8', 'FAM65B', 'LST1', 'AIF1', 'BACH2', 'CXorf57', 'VILL', 'RRP9', 'GPR171', 'SIT1', 'TRPM6', 'NIPSNAP3B', 'LAG3', 'CLEC2D', 'CD69', 'GPR19', 'LRMP', 'PPFIBP1', 'TEP1', 'RNASE6', 'FLVCR2', 'BCL11B', 'FAM174B', 'BFSP1', 'MSC', 'TMEM156', 'MAP9', 'RASSF4', 'SPOCK2', 'CDHR1', 'RRP12', 'SEC31B', 'GZMM', 'PLEKHF1', 'TRPM4', 'LAIR2', 'ZNF135', 'MARCH3', 'SEPT8'] not in index"

In [16]:
pathwayMat.loc["MARCH3",:]

KeyError: 'MARCH3'